In [125]:
from copy import deepcopy
import pygame
from game import Game
from myfile import *
import time
from GUI import *

In [126]:
def simulate_move(piece, move, board, game, skip):
    """
    Simulates a move on the board by moving a piece, capturing any opponent's piece, and updating the game state.
    and return updated game board after the move.
    """
    board.move(piece, move[0], move[1])  # Move the piece on the board.
    if skip:
        board.remove(skip)  # Remove opponent's piece that was captured during the move.

    return board

In [127]:
def get_all_moves(board, color, game):
    """
    Gets all valid moves for a given color on the current game board.
    and returns a List of all possible game boards after making each move.
    """
    moves = []

    for piece in board.get_all_pieces(color):
        valid_moves = board.get_valid_moves(piece)  # Get all valid moves for the piece.
        for move, skip in valid_moves.items():
            temp_board = deepcopy(board)
            temp_piece = temp_board.get_piece(piece.row, piece.col)
            new_board = simulate_move(temp_piece, move, temp_board, game, skip)  # Simulate the move on the board.
            moves.append(new_board)
    
    return moves

In [128]:
def alpha_beta(position, depth, alpha, beta, max_player, game, human_play):
    """
    Implementation of the Alpha-Beta Pruning algorithm over minimax
    """
    if depth == 0:
        # If depth is 0, evaluate the current position and return the score along with the position.
        return position.evaluate(game.board.player), position

    if max_player:
        if human_play == True: 
            if game.turn == "red":
                color_m = "red"  # Determine the color of the opponent's pieces
            else:
                color_m = "white"  # Determine the color of the own pieces
        else:
            if game.turn == "red":
                color_m = "white" 
            else:
                color_m = "red"  
        best_move = None
        for move in get_all_moves(position, color_m, game):
            # Recursively call alpha_beta with the next move and update alpha value.
            evaluation = alpha_beta(move, depth-1, alpha, beta, False, game, human_play)[0]
            if evaluation > alpha:
                alpha = evaluation
                best_move = move
            if alpha >= beta:
                # Perform pruning if alpha is greater than or equal to beta.
                break

        return alpha, best_move
    else:
        best_move = None
        for move in get_all_moves(position, game.turn, game):
            # Recursively call alpha_beta with the next move and update beta value.
            evaluation = alpha_beta(move, depth-1, alpha, beta, True, game, human_play)[0]
            if evaluation < beta:
                beta = evaluation
                best_move = move
            if beta <= alpha:
                # Perform pruning if beta is less than or equal to alpha.
                break

        return beta, best_move

In [129]:
#INITIALLY CHECKING THE ALGORITHM USING THIS
def minimax(position, depth, max_player, game, human_play):
    """
    Implementation of the minimax algorithm for game AI.
    """
    if depth == 0:
        return position.evaluate(game.board.player), position  # Return evaluation of current position and the position itself
    
    if max_player:
        maxEval = float('-inf')  #
        best_move = None 
        if human_play == True: 
            if game.turn == "red":
                color_m = "red"  # Determine the color of the opponent's pieces
            else:
                color_m = "white"  # Determine the color of the own pieces
        else:
            if game.turn == "red":
                color_m = "white" 
            else:
                color_m = "red"  
        for move in get_all_moves(position, color_m, game): 
            evaluation = minimax(move, depth-1, False, game, human_play)[0] 
            maxEval = max(maxEval, evaluation)  # Update the maximum evaluation score
            if maxEval == evaluation:
                best_move = move  # Update the best move if a better move is found
        
        return maxEval, best_move  
    else:
        minEval = float('inf')  
        best_move = None  
        for move in get_all_moves(position, game.turn, game):  
            evaluation = minimax(move, depth-1, True, game, human_play)[0]  
            minEval = min(minEval, evaluation)  # Update the minimum evaluation score
            if minEval == evaluation:
                best_move = move  
        
        return minEval, best_move  # Return the minimum evaluation score and the best move found

In [130]:
#HUMAN VS AI
def human_AI(play1):
    player1 = play1
    player2 = None
    obj = Game(player1, player2)
    start_time = time.time()
    counter = 0
    
    while counter < 40:
        
        old_pieces = obj.board.red_left + obj.board.white_left
        if obj.turn == "red":
            value, new_board = alpha_beta(obj.get_board(), 2, float("-inf"), float("inf"), obj.turn, obj, True)
            #value, new_board = minimax(obj.get_board(), 2, obj.turn, obj, True)
            obj.ai_move(new_board)
            print(new_board.board)
            gui = Checkerboard(obj.board.board)

        else:
            gui = Checkerboard(obj.board.board)
            row, col = gui.run()
            obj.select(row, col)
            
            
        print(obj.turn)
        new_pieces = obj.board.red_left + obj.board.white_left
        difference = old_pieces - new_pieces
        if difference > 0:
            counter = 0
        else:
            counter += 1
        print("DIFF: ", old_pieces - new_pieces)
        winner = obj.winner()

        if winner == "red":
            best_player = obj.player1
        else:
            best_player = obj.player2

    end_time = time.time()
    print("Duration: ", end_time - start_time)

    return new_board, winner, best_player

In [131]:
def AI_vs_AI(player_1, player_2):
    player1 = player_1
    player2 = player_2
    obj = Game(player1, player2)
    start_time = time.time()
    counter = 0
    while counter < 40:
        old_pieces = obj.board.red_left + obj.board.white_left
        # old_pieces = len(obj.board.get_all_pieces(obj.turn)) + len(obj.board.get_all_pieces(opponent))
        value, new_board = alpha_beta(obj.get_board(), 2, float("-inf"), float("inf"), obj.turn, obj, False)
        #value, new_board = minimax(obj.get_board(), 2, obj.turn, obj, False)
        #print(obj.turn)
        # print(new_board.board)
        obj.ai_move(new_board)
        new_pieces = obj.board.red_left + obj.board.white_left
        difference = old_pieces - new_pieces
        if difference > 0:
            counter = 0
        else:
            counter += 1
        # print(counter)
        # print("DIFF: ", old_pieces - new_pieces)
        winner = obj.winner()

        if winner == "red":
            best_player = obj.player1
        else:
            best_player = obj.player2

    end_time = time.time()
    #print("Duration: ", end_time - start_time)

    return new_board, winner, best_player

In [132]:
def parentSelection(chromosomes, type):

    if type == "random":
        index = np.random.choice(chromosomes, 2, replace=False) 
        
        parent1 = index[0]
        parent2 = index[1]

        if parent1 == parent2:
            print("Parents are the same??")

    elif type == "truncation":
        sorted_population = sorted(chromosomes, key=lambda chromosome: chromosome.score, reverse=True)
        parent1 = sorted_population[-1]
        parent2 = sorted_population[-2]

    elif type == "fps":
        total_fitness = sum(chromosome.score for chromosome in chromosomes)
        chromosome_probabilities = [chromosome.fitness/total_fitness for chromosome in chromosomes]
        
        parent1 = random.choices(chromosomes, weights=chromosome_probabilities)
        parent2 = random.choices(chromosomes, weights=chromosome_probabilities)
       

    elif type == "binary_tournament":
        parent_1 = random.choice(chromosomes)
        parent_2 = random.choice(chromosomes)

        # Choose the fittest chromosome as the first parent
        if parent_1.score > parent_2.score:
            parent1 = parent_1
        else:
            parent1 = parent_2

        # repeats for second parent
        parent_1 = random.choice(chromosomes)
        parent_2 = random.choice(chromosomes)
        if parent_1.score > parent_2.score:
            parent2 = parent_1
        else:
            parent2 = parent_2

    return parent1, parent2

In [133]:
def crossover(parent1, parent2):

    p1first_layer_weights, p1first_layer_bias, p1second_layer_weights, p1second_layer_bias, p1third_layer_weights,  p1third_layer_bias = parent1.getWeights()
    p2first_layer_weights, p2first_layer_bias, p2second_layer_weights, p2second_layer_bias, p2third_layer_weights,  p2third_layer_bias = parent2.getWeights()
    
    crossoverPoint1 = random.randint(0, len(p1first_layer_weights) - 1)

    o1first_layer_weights = []
    o2first_layer_weights = []

    for i in range(len(p1first_layer_weights)):
        
        o1 = np.concatenate((p1first_layer_weights[i][:crossoverPoint1],(p2first_layer_weights[i][crossoverPoint1:])))
        o2 = np.concatenate((p2first_layer_weights[i][:crossoverPoint1], p1first_layer_weights[i][crossoverPoint1:]))

        o1first_layer_weights.append(o1)
        o2first_layer_weights.append(o2)

    crossoverPoint2 = random.randint(0, len(p1first_layer_bias) - 1)
   
    o1first_layer_bias = []
    o2first_layer_bias = []

    for i in range(len(p1first_layer_bias)):

        o1 = np.concatenate((p1first_layer_bias[i][:crossoverPoint2], p2first_layer_bias[i][crossoverPoint2:]))
        o2 = np.concatenate((p2first_layer_bias[i][:crossoverPoint2], p1first_layer_bias[i][crossoverPoint2:]))

        o1first_layer_bias.append(o1)
        o2first_layer_bias.append(o2)

    crossoverPoint3 = random.randint(0, len(p1second_layer_weights) - 1)
   
    o1second_layer_weights = []
    o2second_layer_weights = []

    for i in range(len(p1second_layer_weights)):

        o1 = np.concatenate((p1second_layer_weights[i][:crossoverPoint3], p2second_layer_weights[i][crossoverPoint3:]))
        o2 = np.concatenate((p2second_layer_weights[i][:crossoverPoint3], p1second_layer_weights[i][crossoverPoint3:]))

        o1second_layer_weights.append(o1)
        o2second_layer_weights.append(o2)

    crossoverPoint4 = random.randint(0, len(p1second_layer_bias) - 1)

    o1second_layer_bias = []
    o2second_layer_bias = []

    for i in range(len(p1second_layer_bias)):

        o1 = np.concatenate((p1second_layer_bias[i][:crossoverPoint4], p2second_layer_bias[i][crossoverPoint4:]))
        o2 = np.concatenate((p2second_layer_bias[i][:crossoverPoint4], p1second_layer_bias[i][crossoverPoint4:]))

        o1second_layer_bias.append(o1)
        o2second_layer_bias.append(o2)

    crossoverPoint5 = random.randint(0, len(p1third_layer_weights) - 1)

    o1third_layer_weights = []
    o2third_layer_weights = []

    for i in range(len(p1third_layer_weights)):

        o1 = np.concatenate((p1third_layer_weights[i][:crossoverPoint5], p2third_layer_weights[i][crossoverPoint5:]))
        o2 = np.concatenate((p2third_layer_weights[i][:crossoverPoint5], p1third_layer_weights[i][crossoverPoint5:]))

        o1third_layer_weights.append(o1)
        o2third_layer_weights.append(o2)

    crossoverPoint6 = random.randint(0, len(p1third_layer_bias) - 1)

    o1third_layer_bias = []
    o2third_layer_bias = []

    for i in range(len(p1third_layer_bias)):

        o1 = np.concatenate((p1third_layer_bias[0][:crossoverPoint6], p2third_layer_bias[0][crossoverPoint6:]))
        o2 = np.concatenate((p2third_layer_bias[0][:crossoverPoint6], p1third_layer_bias[0][crossoverPoint6:]))

        o1third_layer_bias.append(o1)
        o2third_layer_bias.append(o2)

    offspring1 = []
    offspring1.append(o1first_layer_bias)
    offspring1.append(o1second_layer_bias)
    offspring1.append(o1third_layer_bias)

    offspring1.append(o1first_layer_weights)
    offspring1.append(o1second_layer_weights)
    offspring1.append(o1third_layer_weights)

    offspring2 = []
    offspring2.append(o2first_layer_bias)
    offspring2.append(o2second_layer_bias)
    offspring2.append(o2third_layer_bias)

    offspring2.append(o2first_layer_weights)
    offspring2.append(o2second_layer_weights)
    offspring2.append(o2third_layer_weights)

    return offspring1, offspring2

In [134]:
def uniform_crossover(parent1, parent2):

    p1_weights_biases = parent1.getWeights()
    p2_weights_biases = parent2.getWeights()

    o1_weights_biases = []
    o2_weights_biases = []

    for p1_layer, p2_layer in zip(p1_weights_biases, p2_weights_biases):
        mask = np.random.randint(0, 2, p1_layer.shape).astype(bool)
        o1_layer = np.where(mask, p1_layer, p2_layer)
        o2_layer = np.where(mask, p2_layer, p1_layer)

        o1_weights_biases.append(o1_layer)
        o2_weights_biases.append(o2_layer)

    #offspring1 = o1_weights_biases
    #offspring2 = o2_weights_biases

    p1first_layer_weights, p1first_layer_bias, p1second_layer_weights, p1second_layer_bias, p1third_layer_weights,  p1third_layer_bias = o1_weights_biases
    p2first_layer_weights, p2first_layer_bias, p2second_layer_weights, p2second_layer_bias, p2third_layer_weights,  p2third_layer_bias = o2_weights_biases
    
    offspring1 = []
    offspring1.append(p1first_layer_bias)
    offspring1.append(p1second_layer_bias)
    offspring1.append(p1third_layer_bias)

    offspring1.append(p1first_layer_weights)
    offspring1.append(p1second_layer_weights)
    offspring1.append(p1third_layer_weights)

    offspring2 = []
    offspring2.append(p2first_layer_bias)
    offspring2.append(p2second_layer_bias)
    offspring2.append(p2third_layer_bias)

    print(len(p1second_layer_bias))

    offspring2.append(p2first_layer_weights)
    offspring2.append(p2second_layer_weights)
    offspring2.append(p2third_layer_weights)

    return offspring1, offspring2


In [135]:
def mutationNew(offspring1, offspring2, mutationRate):

    o1first_layer_bias, o1second_layer_bias, o1third_layer_bias, o1first_layer_weights, o1second_layer_weights, o1third_layer_weights = offspring1
    o2first_layer_bias, o2second_layer_bias, o2third_layer_bias, o2first_layer_weights, o2second_layer_weights, o2third_layer_weights = offspring2
    
    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1first_layer_bias[0]) - 1, size=10)

        for i in range(10):
             o1first_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2first_layer_bias[0]) - 1, size=10)


        for i in range(10):
            o2first_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1second_layer_bias[0]) - 1, size=10)

        for i in range(10):
             o1second_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2second_layer_bias[0]) - 1, size=10)


        for i in range(10):
            o2second_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1third_layer_bias[0]) - 1, size=10)

        for i in range(10):
             o1third_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2third_layer_bias[0]) - 1, size=10)


        for i in range(10):
            o2third_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1first_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1first_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o1first_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2first_layer_weights[0]) - 1, size=10)


            for j in range(10):
                o2first_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1second_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1second_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o1second_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2second_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o2second_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1third_layer_weights) - 1):
         if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1third_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o1third_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2third_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o2third_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
    mutatedOffspring1 = []
    mutatedOffspring1.append(o1first_layer_bias)
    mutatedOffspring1.append(o1second_layer_bias)
    mutatedOffspring1.append(o1third_layer_bias)

    mutatedOffspring1.append(o1first_layer_weights)
    mutatedOffspring1.append(o1second_layer_weights)
    mutatedOffspring1.append(o1third_layer_weights)

    mutatedOffspring2 = []
    mutatedOffspring2.append(o2first_layer_bias)
    mutatedOffspring2.append(o2second_layer_bias)
    mutatedOffspring2.append(o2third_layer_bias)

    mutatedOffspring2.append(o2first_layer_weights)
    mutatedOffspring2.append(o2second_layer_weights)
    mutatedOffspring2.append(o2third_layer_weights)

    return mutatedOffspring1, mutatedOffspring2

In [136]:
def mutation(offspring1, offspring2, mutationRate):

    o1first_layer_bias, o1second_layer_bias, o1third_layer_bias, o1first_layer_weights, o1second_layer_weights, o1third_layer_weights = offspring1
    o2first_layer_bias, o2second_layer_bias, o2third_layer_bias, o2first_layer_weights, o2second_layer_weights, o2third_layer_weights = offspring2
    
    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1first_layer_bias[0]) - 1, size=2)
        o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
        

        rand_nums = np.random.randint(0, len(o2first_layer_bias[0]) - 1, size=2)
        o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1second_layer_bias[0]) - 1, size=2)
        o1second_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1second_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
        
        rand_nums = np.random.randint(0, len(o2second_layer_bias[0]) - 1, size=2)
        o2second_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2second_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1third_layer_bias[0]) - 1, size=2)
        o1third_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1third_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2third_layer_bias[0]) - 1, size=2)
        o2third_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2third_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1first_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1first_layer_weights[0]) - 1, size=2)
            o1first_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1first_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2first_layer_weights[0]) - 1, size=2)
            o2first_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2first_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1second_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1second_layer_weights[0]) - 1, size=2)
            o1second_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1second_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2second_layer_weights[0]) - 1, size=2)
            o2second_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2second_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1third_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1third_layer_weights[0]) - 1, size=2)
            o1third_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1third_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2third_layer_weights[0]) - 1, size=2)
            o2third_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2third_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

    mutatedOffspring1 = []
    mutatedOffspring1.append(o1first_layer_bias)
    mutatedOffspring1.append(o1second_layer_bias)
    mutatedOffspring1.append(o1third_layer_bias)

    mutatedOffspring1.append(o1first_layer_weights)
    mutatedOffspring1.append(o1second_layer_weights)
    mutatedOffspring1.append(o1third_layer_weights)

    mutatedOffspring2 = []
    mutatedOffspring2.append(o2first_layer_bias)
    mutatedOffspring2.append(o2second_layer_bias)
    mutatedOffspring2.append(o2third_layer_bias)

    mutatedOffspring2.append(o2first_layer_weights)
    mutatedOffspring2.append(o2second_layer_weights)
    mutatedOffspring2.append(o2third_layer_weights)

    return mutatedOffspring1, mutatedOffspring2

In [137]:
def calculateFitness(population):

    for i in range(len(population) - 1):

        player1 = population[i]

        challengers = population[:i] +  population[i+1:]

        randomPlayers = np.random.choice(challengers, size=3, replace=False)

        for player2 in randomPlayers:
            obj = Game(player1, player2)
            new_board, winner, best_player = AI_vs_AI(player1, player2)

            if winner=="draw":
                player1.score +=1
                player2.score +=1
            else:
                best_player.score +=2

                if winner == 'red':
                    player1.score -= 1
                else:
                    player2.score -= 1

            #print("game complete")

In [138]:
def survivorSelection(newPopulation, type):

    finalPopulation = []
        
    if type == "truncation":

        tup = []
        for i in newPopulation:
            tup.append((i.score, i))
        
        sortedlist = sorted(tup, key=lambda x: x[0], reverse=True)

        for pop in sortedlist:
            finalPopulation.append(pop[1])

        finalPopulation =  finalPopulation[:10]
    
    elif type == "fps":
        total_fitness = sum(chromosome.score for chromosome in newPopulation)
        chromosome_probabilities = [chromosome.fitness/total_fitness for chromosome in newPopulation]
        for i in range(10):
            finalPopulation.append(np.random.choice(newPopulation, p=chromosome_probabilities))


    elif type == "binary_tournament":
         
        for i in range(10):
        # Choose two random parents from the population
            parent1 = random.choice(newPopulation)
            parent2 = random.choice(newPopulation)

            # Choose the fittest chromosome as the first parent
            if parent1.score > parent2.score:
                finalPopulation.append(parent1)
            else:
                finalPopulation.append(parent2)
    
    return finalPopulation

In [139]:
def NueroEvolution(population, generations):

    best_overall_player_score = 0

    chromosomes = []

    #Initialise random population
    for i in range(population):
        chromosomes.append(evolutionary_player(i))

    calculateFitness(chromosomes)

    for i in chromosomes:
        print(i.score)
        print(i.first_layer_weights)

    #print(chromosomes)
    count = 10
    
    for n in range(generations):
        print("Generation: ", n)

        parent1, parent2 = parentSelection(chromosomes, "random")

        print("Parent 1", parent1)
        print("Parent 2", parent2)

        offspring1, offspring2 = crossover(parent1, parent2)  
        print("Crossover complete")

        offspring1AfterMutation, offspring2AfterMutation = mutation(offspring1, offspring2, mutationRate=0.5)

        parent1player, parent2player = createNeuralNetwork(offspring1AfterMutation, offspring2AfterMutation, count)
        print("Mutation complete")

        newPopulation = chromosomes + [parent1player, parent2player]
        

        calculateFitness(newPopulation)
        print("Fitness Calculation complete")

        chromosomes = survivorSelection(newPopulation, "truncation")

        high = 0
        
        print("Generation Scores")
        for i in chromosomes:
            if i.score > high:
                high = i.score
                highchrome = i
            print(i.score)
        
        print("Best Score", high, "Name", highchrome)

        if highchrome == parent1 or highchrome == parent2:
            print("equal??")

        if high > best_overall_player_score:
            best_overall_player_score = high
            best_overall_player = highchrome

    


        count += 2

    return best_overall_player

In [140]:
#player = NueroEvolution(12, 1)

In [141]:
#new_board, winner, best_player = human_AI(player)
#new_board, winner, best_player = AI_vs_AI()
'''
for i in new_board.board:
    print(i)
for i in new_board.board:
    for piece in i:
        if piece !=0:
            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)
print("WINNER COLOR:")
print(winner)
print("WINNER PLAYER NUM:")
print(best_player.number)
print("best first layer weights:")
print(best_player.first_layer_weights)
print("best first layer bias:")
print(best_player.first_layer_bias)
print("best second layer weights:")
print(best_player.second_layer_weights)
print("best second layer bias:")
print(best_player.second_layer_bias)
print("best third layer weights:")
print(best_player.third_layer_weights)
print("best third layer bias:")
print(best_player.third_layer_bias)'''

'\nfor i in new_board.board:\n    print(i)\nfor i in new_board.board:\n    for piece in i:\n        if piece !=0:\n            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)\nprint("WINNER COLOR:")\nprint(winner)\nprint("WINNER PLAYER NUM:")\nprint(best_player.number)\nprint("best first layer weights:")\nprint(best_player.first_layer_weights)\nprint("best first layer bias:")\nprint(best_player.first_layer_bias)\nprint("best second layer weights:")\nprint(best_player.second_layer_weights)\nprint("best second layer bias:")\nprint(best_player.second_layer_bias)\nprint("best third layer weights:")\nprint(best_player.third_layer_weights)\nprint("best third layer bias:")\nprint(best_player.third_layer_bias)'

In [142]:
def NueroEvolution2(population, generations):

    best_overall_player_score = 0

    chromosomes = []

    #Initialise random population
    for i in range(population):
        chromosomes.append(evolutionary_player(i))

    calculateFitness(chromosomes)

   # for i in chromosomes:
    #    print(i.score)
    #    print(i.first_layer_weights)

    #print(chromosomes)
    count = 10
    
    for n in range(generations):
        print("Generation: ", n)

        parent1, parent2 = parentSelection(chromosomes, "random")

        print("Parent 1", parent1)
        print("Parent 2", parent2)

        offspring1, offspring2 = uniform_crossover(parent1, parent2)  
        print("Crossover complete")

        offspring1AfterMutation, offspring2AfterMutation = mutationNew(offspring1, offspring2, mutationRate=0.5)

        parent1player, parent2player = createNeuralNetwork(offspring1AfterMutation, offspring2AfterMutation, count)
        print("Mutation complete")

        newPopulation = chromosomes + [parent1player, parent2player]
        

        calculateFitness(newPopulation)
        print("Fitness Calculation complete")

        chromosomes = survivorSelection(newPopulation, "truncation")

        high = 0
        
        print("Generation Scores")
        for i in chromosomes:
            if i.score > high:
                high = i.score
                highchrome = i
            print(i.score)
        
        print("Best Score", high, "Name", highchrome)

        if highchrome == parent1 or highchrome == parent2:
            print("equal??")

        if high > best_overall_player_score:
            best_overall_player_score = high
            best_overall_player = highchrome

    


        count += 2

    return best_overall_player

In [143]:
player2 = NueroEvolution2(12, 20)

Generation:  0
Parent 1 <myfile.Evol_Player object at 0x141b9fdd0>
Parent 2 <myfile.Evol_Player object at 0x141b9d350>
1
Crossover complete
Mutation complete
Fitness Calculation complete
Generation Scores
11
8
7
7
7
6
6
6
6
5
Best Score 11 Name <myfile.Evol_Player object at 0x141b9d350>
equal??
Generation:  1
Parent 1 <myfile.Evol_Player object at 0x141b9fdd0>
Parent 2 <myfile.Evol_Player object at 0x1414baf50>
1
Crossover complete
Mutation complete
Fitness Calculation complete
Generation Scores
13
12
10
10
10
10
10
9
8
8
Best Score 13 Name <myfile.Evol_Player object at 0x141b9fdd0>
equal??
Generation:  2
Parent 1 <myfile.Evol_Player object at 0x141b9fdd0>
Parent 2 <myfile.Evol_Player object at 0x141b9cd10>
1
Crossover complete
Mutation complete
Fitness Calculation complete
Generation Scores
16
16
15
15
14
14
13
11
10
10
Best Score 16 Name <myfile.Evol_Player object at 0x141b9fdd0>
equal??
Generation:  3
Parent 1 <myfile.Evol_Player object at 0x141b9d350>
Parent 2 <myfile.Evol_Player o

In [144]:
new_board, winner, best_player = human_AI(player2)
#new_board, winner, best_player = AI_vs_AI()

for i in new_board.board:
    print(i)
for i in new_board.board:
    for piece in i:
        if piece !=0:
            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)
print("WINNER COLOR:")
print(winner)
print("WINNER PLAYER NUM:")
print(best_player.number)
print("best first layer weights:")
print(best_player.first_layer_weights)
print("best first layer bias:")
print(best_player.first_layer_bias)
print("best second layer weights:")
print(best_player.second_layer_weights)
print("best second layer bias:")
print(best_player.second_layer_bias)
print("best third layer weights:")
print(best_player.third_layer_weights)
print("best third layer bias:")

you have selected the piece at (2,4)
white
DIFF:  0
you have selected the piece at (2,5)
white
DIFF:  0
you have selected the piece at (3,4)
red
DIFF:  0
[[0, white, 0, white, 0, white, 0, white], [white, 0, white, 0, white, 0, white, 0], [0, white, 0, white, 0, 0, 0, white], [0, 0, 0, 0, white, 0, 0, 0], [0, red, 0, 0, 0, 0, 0, 0], [0, 0, red, 0, red, 0, red, 0], [0, red, 0, red, 0, red, 0, red], [red, 0, red, 0, red, 0, red, 0]]
white
DIFF:  0
you have selected the piece at (2,7)
white
DIFF:  0
you have selected the piece at (3,6)
red
DIFF:  0
[[0, white, 0, white, 0, white, 0, white], [white, 0, white, 0, white, 0, white, 0], [0, white, 0, white, 0, 0, 0, 0], [0, 0, 0, 0, white, 0, white, 0], [0, red, 0, 0, 0, red, 0, 0], [0, 0, red, 0, red, 0, 0, 0], [0, red, 0, red, 0, red, 0, red], [red, 0, red, 0, red, 0, red, 0]]
white
DIFF:  0
you have selected the piece at (3,4)
white
DIFF:  0
you have selected the piece at (5,6)
red
DIFF:  1
[[0, white, 0, white, 0, white, 0, white], [white,

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
new_board, winner, best_player = human_AI(player2)
#new_board, winner, best_player = AI_vs_AI()

for i in new_board.board:
    print(i)
for i in new_board.board:
    for piece in i:
        if piece !=0:
            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)
print("WINNER COLOR:")
print(winner)
print("WINNER PLAYER NUM:")
print(best_player.number)
print("best first layer weights:")
print(best_player.first_layer_weights)
print("best first layer bias:")
print(best_player.first_layer_bias)
print("best second layer weights:")
print(best_player.second_layer_weights)
print("best second layer bias:")
print(best_player.second_layer_bias)
print("best third layer weights:")
print(best_player.third_layer_weights)
print("best third layer bias:")